## <div style ="font-size:25px; border-radius:25px; border:3666; padding:10px;  background-color:#fffafa; text-align:center; color:#000000;">CNN ImageNet</div>


<img src="https://neurohive.io/wp-content/uploads/2018/10/AlexNet-1.png">

In [1]:
train_path = '/kaggle/input/cats-and-dogs/train'
test_path = '/kaggle/input/cats-and-dogs/val'

In [2]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.applications import Xception

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,               
    rotation_range=20,             
    width_shift_range=0.2,         
    height_shift_range=0.2,         
    horizontal_flip=True,          
    fill_mode='nearest'             
)

In [4]:
images_size = 200
batch_size = 32

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_path,                    
    target_size=(images_size, images_size), 
    batch_size=batch_size,          
    seed=32,                        
    shuffle=True,                    
    class_mode='categorical'        
)

Found 275 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [7]:
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(images_size, images_size),
    batch_size = batch_size,
    class_mode='categorical')

Found 70 images belonging to 2 classes.


In [8]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(images_size, images_size, 3))

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
for layer in base_model.layers:
    layer.trainable = False

In [10]:
model = tf.keras.models.Sequential([
    base_model,
    
    layers.Flatten(),
    
    layers.Dense(128,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2,activation='softmax'),
])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ xception (Functional)           │ ?                      │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,861,480 (79.58 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 20,861,480 (79.58 MB)

In [11]:
model.compile(optimizer='Adam',
             loss="categorical_crossentropy",
              metrics=['accuracy']
             )

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=7,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    
)

Epoch 1/7


I0000 00:00:1727522045.777746      84 service.cc:145] XLA service 0x7c62e0008ec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727522045.777800      84 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1727522045.777804      84 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


1/8 ━━━━━━━━━━━━━━━━━━━━ 2:09 18s/step - accuracy: 0.6250 - loss: 0.8187

I0000 00:00:1727522056.497780      84 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8/8 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.6997 - loss: 8.9464 - val_accuracy: 1.0000 - val_loss: 0.0060
Epoch 2/7
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 493ms/step - accuracy: 0.9062 - loss: 1.0301 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/7
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 167ms/step - accuracy: 0.9619 - loss: 0.9443 - val_accuracy: 1.0000 - val_loss: 0.0085
Epoch 4/7
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9062 - loss: 0.6928 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/7
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 171ms/step - accuracy: 0.9234 - loss: 0.2841 - val_accuracy: 1.0000 - val_loss: 1.5460e-07
Epoch 6/7
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0091 - val_accuracy: 1.0000 - val_loss: 0.0036
Epoch 7/7
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 144ms/step - accuracy: 0.9879 - loss: 0.0399 - val_accuracy: 1.0000 - val_loss: 4.1122e-06


In [14]:
# Evaluate on test dataset
score = model.evaluate(test_generator, verbose=False)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 3.7597326354443794e-06
Test accuracy: 1.0
